In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import tensorflow as tf
from malaya_speech.train.model import fast_swave, fastspeech
import malaya_speech.augmentation.waveform as augmentation
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
from glob import glob

wavs = glob('../speech/example-speaker/*.wav')
len(wavs)

8

In [5]:
import random

sr = 22050
speakers_size = 4

def read_wav(f):
    return malaya_speech.load(f, sr = sr)

def random_sampling(s, length):
    return augmentation.random_sampling(s, sr = sr, length = length)

def combine_speakers(files, n = 5, limit = 4):
    w_samples = random.sample(files, n)
    w_samples = [
        random_sampling(
            read_wav(f)[0],
            length = min(
                random.randint(10000 // n, 20000 // n), 10000
            ),
        )
        for f in w_samples
    ]
    y = [w_samples[0]]
    left = w_samples[0].copy() * random.uniform(0.5, 1.0)
    start, end = [], []
    start.append(0)
    end.append(len(left))

    combined = None

    for i in range(1, n):
        right = w_samples[i].copy() * random.uniform(0.5, 1.0)
        overlap = random.uniform(0.1, 0.9)
        print(i, overlap, len(right))
        len_overlap = int(overlap * len(right))
        minus = len(left) - len_overlap
        padded_right = np.pad(right, (minus, 0))
        start.append(minus)
        end.append(len(padded_right))
        left = np.pad(left, (0, len(padded_right) - len(left)))

        left = left + padded_right

        if i >= (limit - 1):
            if combined is None:
                combined = padded_right
            else:
                combined = np.pad(
                    combined, (0, len(padded_right) - len(combined))
                )
                combined += padded_right

        else:
            y.append(padded_right)

    if combined is not None:
        y.append(combined)

    for i in range(len(y)):
        if len(y[i]) != len(left):
            y[i] = np.pad(y[i], (0, len(left) - len(y[i])))
            y[i] = y[i] / np.max(np.abs(y[i]))

    left = left / np.max(np.abs(left))
    
    while len(y) < limit:
        y.append(np.zeros((len(left))))
        start.append(0)
        end.append(0)
        
    return left, y, start[:limit], end[:limit]

# y, _ = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav')
# y = np.expand_dims(y, 0).astype(np.float32)
# y.shape

In [8]:
left, y, start, end = combine_speakers(wavs, random.randint(1, len(wavs)))
len(left) / sr, len(y), start, end

1 0.8012907465426807 73436
2 0.10942690111288762 102520
3 0.4560287740057154 59334


(9.784943310657596,
 4,
 [0, 18744, 80962, 156424],
 [77587, 92180, 183482, 215758])

In [9]:
left_mel = malaya_speech.featurization.universal_mel(left)
y_mel = [malaya_speech.featurization.universal_mel(i) for i in y]

In [10]:
config = malaya_speech.config.fastspeech_config
dim = malaya_speech.config.fastspeech_config['encoder_hidden_size']
config['encoder_num_hidden_layers'] = 2
config = fastspeech.Config(vocab_size = 1, **config)

In [11]:
model = fast_swave.Model(config, R = 2, C = speakers_size, sample_rate = sr, N = dim, H = dim)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
X = tf.placeholder(tf.float32, [None, None, 80])
Y = tf.placeholder(tf.float32, [None, speakers_size, None, 80])
lengths = tf.placeholder(tf.int32, [None])
outputs, output_all = model(X)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
reduction_indices is deprecated, use axis instead


In [11]:
output_all, outputs

([<tf.Tensor 'fast-swave/Separator/transpose_3:0' shape=(?, ?, 1536) dtype=float32>],
 [<tf.Tensor 'fast-swave/mel_before/BiasAdd:0' shape=(?, ?, 4, 80) dtype=float32>])

In [12]:
outputs

[<tf.Tensor 'fast-swave/mel_before/BiasAdd:0' shape=(?, ?, 4, 80) dtype=float32>]

In [13]:
loss = 0
for c_idx, est_src in enumerate(outputs):
    coeff = ((c_idx + 1) * (1 / len(outputs)))
    abs_loss = fast_swave.calculate_loss(Y, est_src, lengths, C = speakers_size)
    loss += (coeff * abs_loss)
loss /= len(outputs)

Tensor("Placeholder_1:0", shape=(?, 4, ?, 80), dtype=float32) Tensor("mul:0", shape=(?, 4, ?, 80), dtype=float32)


In [14]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [15]:
%%time

o = sess.run(outputs, feed_dict = {X: [left_mel]})

CPU times: user 8.51 s, sys: 743 ms, total: 9.25 s
Wall time: 1.76 s


In [16]:
o[0].shape, left_mel.shape

((1, 936, 4, 80), (936, 80))

In [17]:
np.array(y_mel).shape

(4, 936, 80)

In [18]:
sess.run(loss, feed_dict = {X: [left_mel, left_mel], Y: [y_mel, y_mel], lengths: [len(left_mel),
                                                                                 len(left_mel)]})

5.0871363

In [19]:
saver = tf.train.Saver()

In [20]:
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [21]:
!ls -lh test

total 465072
-rw-r--r--  1 huseinzolkepli  staff    77B Mar  3 21:08 checkpoint
-rw-r--r--  1 huseinzolkepli  staff   187M Mar  3 21:08 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    12K Mar  3 21:08 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff    27M Mar  3 21:08 model.ckpt.meta


In [22]:
!rm -rf test